In [ ]:
!pip install requests

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import csv
import re

In [ ]:
## Dados de audiências públicas

# Defina os parâmetros da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx/AudienciasPublicas"
params = {
    "dataInicial": "2021-01-01",  # Substitua pela data inicial desejada
    "dataFinal": "2024-07-15"     # Substitua pela data final desejada
}

# Envie a solicitação GET
response = requests.get(url, params=params)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

    # Defina o caminho para o arquivo CSV de saída
    output_csv = 'audiencias.csv'

    # Abra o arquivo CSV para escrita
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escreva o cabeçalho
        writer.writerow([
            'Chave', 'Numero', 'Tipo', 'Comissao', 'Data', 'Horario', 
            'Local', 'Tema', 'TipoMateria', 'Sigla', 'NumeroMateria', 
            'Ano', 'Ementa'
        ])

        # Extraia dados do XML e escreva no CSV
        for audiencia in root.findall("{http://www.camara.sp.gov.br/}Audiencia"):
            chave = audiencia.find("{http://www.camara.sp.gov.br/}Chave").text
            numero = audiencia.find("{http://www.camara.sp.gov.br/}Numero").text
            tipo = audiencia.find("{http://www.camara.sp.gov.br/}Tipo").text
            comissao = audiencia.find("{http://www.camara.sp.gov.br/}Comissao").text
            data = audiencia.find("{http://www.camara.sp.gov.br/}Data").text
            horario = audiencia.find("{http://www.camara.sp.gov.br/}Horario").text
            local = audiencia.find("{http://www.camara.sp.gov.br/}Local").text
            tema = audiencia.find("{http://www.camara.sp.gov.br/}Tema")

            for materia in audiencia.findall("{http://www.camara.sp.gov.br/}Projetos/{http://www.camara.sp.gov.br/}Materia"):
                tipo_materia = materia.find("{http://www.camara.sp.gov.br/}Tipo").text
                sigla = materia.find("{http://www.camara.sp.gov.br/}Sigla").text
                numero_materia = materia.find("{http://www.camara.sp.gov.br/}Numero").text
                ano = materia.find("{http://www.camara.sp.gov.br/}Ano").text
                ementa = materia.find("{http://www.camara.sp.gov.br/}Ementa").text
                
                # Escreva a linha no CSV
                writer.writerow([
                    chave, numero, tipo, comissao, data, horario, local, tema, 
                    tipo_materia, sigla, numero_materia, ano, ementa
                ])
else:
    print(f"Falha na solicitação: {response.status_code}")


In [ ]:
## Dados sobre projetos de lei

# Defina a URL e os cabeçalhos da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.camara.sp.gov.br/CatalogoDeAssuntos"
}

# Corpo da solicitação SOAP
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <CatalogoDeAssuntos xmlns="http://www.camara.sp.gov.br/" />
  </soap:Body>
</soap:Envelope>"""

# Envie a solicitação SOAP
response = requests.post(url, headers=headers, data=body)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

    # Defina o caminho para o arquivo CSV de saída
    output_csv = 'catalogo_de_assuntos.csv'

    # Abra o arquivo CSV para escrita
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escreva o cabeçalho
        writer.writerow([
            'Chave', 'Texto', 'Nota', 'Autorizado', 'ChaveUse', 'Use'
        ])

        # Extraia dados do XML e escreva no CSV
        namespace = "{http://www.camara.sp.gov.br/}"
        for assunto in root.findall(f".//{namespace}Assunto"):
            chave = assunto.find(f"{namespace}Chave").text
            texto = assunto.find(f"{namespace}Texto").text
            nota = assunto.find(f"{namespace}Nota")
            autorizado = assunto.find(f"{namespace}Autorizado").text
            chave_use = assunto.find(f"{namespace}ChaveUse")
            use = assunto.find(f"{namespace}Use")

            # Escreva a linha no CSV
            writer.writerow([chave, texto, nota, autorizado, chave_use, use])
else:
    print(f"Falha na solicitação: {response.status_code}")

In [ ]:
## Dados de sessões plenárias 2023

# Defina a URL e os cabeçalhos da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.camara.sp.gov.br/PautasSessoesPlenarias"
}

# Corpo da solicitação SOAP
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <PautasSessoesPlenarias xmlns="http://www.camara.sp.gov.br/">
      <Ano>2023</Ano> <!-- Substitua pelo ano desejado -->
    </PautasSessoesPlenarias>
  </soap:Body>
</soap:Envelope>"""

# Envie a solicitação SOAP
response = requests.post(url, headers=headers, data=body)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

    # Salvar o conteúdo XML da resposta em um arquivo
    with open('pauta23.xml', 'wb') as file:
        file.write(response.content)

    # Defina o caminho para o arquivo CSV de saída
    output_csv = 'pautas_sessoes_plenarias23.csv'

    # Abra o arquivo CSV para escrita
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escreva o cabeçalho
        writer.writerow([
            'Chave', 'Sessao', 'Data', 'Oradores', 'Status'
        ])

        # Extraia dados do XML e escreva no CSV
        namespace = "{http://www.camara.sp.gov.br/}"
        for pauta in root.findall(f".//{namespace}Pauta"):
            chave = pauta.find(f"{namespace}Chave").text
            sessao = pauta.find(f"{namespace}Sessao").text
            data = pauta.find(f"{namespace}Data").text
            oradores = pauta.find(f"{namespace}Oradores").text
            status = pauta.find(f"{namespace}Status").text

            # Escreva a linha no CSV
            writer.writerow([chave, sessao, data, oradores, status])
else:
    print(f"Falha na solicitação: {response.status_code}")

In [ ]:
## Dados sessões plenárias 2024

# Defina a URL e os cabeçalhos da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.camara.sp.gov.br/PautasSessoesPlenarias"
}

# Corpo da solicitação SOAP
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <PautasSessoesPlenarias xmlns="http://www.camara.sp.gov.br/">
      <Ano>2024</Ano> <!-- Substitua pelo ano desejado -->
    </PautasSessoesPlenarias>
  </soap:Body>
</soap:Envelope>"""

# Envie a solicitação SOAP
response = requests.post(url, headers=headers, data=body)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

        # Salvar o conteúdo XML da resposta em um arquivo
    with open('pauta24.xml', 'wb') as file:
        file.write(response.content)

    # Defina o caminho para o arquivo CSV de saída
    output_csv = 'pautas_sessoes_plenarias24.csv'

    # Abra o arquivo CSV para escrita
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escreva o cabeçalho
        writer.writerow([
            'Chave', 'Sessao', 'Data', 'Oradores', 'Status'
        ])

        # Extraia dados do XML e escreva no CSV
        namespace = "{http://www.camara.sp.gov.br/}"
        for pauta in root.findall(f".//{namespace}Pauta"):
            chave = pauta.find(f"{namespace}Chave").text
            sessao = pauta.find(f"{namespace}Sessao").text
            data = pauta.find(f"{namespace}Data").text
            oradores = pauta.find(f"{namespace}Oradores").text
            status = pauta.find(f"{namespace}Status").text

            # Escreva a linha no CSV
            writer.writerow([chave, sessao, data, oradores, status])
else:
    print(f"Falha na solicitação: {response.status_code}")

In [ ]:
## Dados sessões plenárias 2022

# Defina a URL e os cabeçalhos da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.camara.sp.gov.br/PautasSessoesPlenarias"
}

# Corpo da solicitação SOAP
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <PautasSessoesPlenarias xmlns="http://www.camara.sp.gov.br/">
      <Ano>2022</Ano> <!-- Substitua pelo ano desejado -->
    </PautasSessoesPlenarias>
  </soap:Body>
</soap:Envelope>"""

# Envie a solicitação SOAP
response = requests.post(url, headers=headers, data=body)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

        # Salvar o conteúdo XML da resposta em um arquivo
    with open('pauta22.xml', 'wb') as file:
        file.write(response.content)

    # Defina o caminho para o arquivo CSV de saída
    output_csv = 'pautas_sessoes_plenarias22.csv'

    # Abra o arquivo CSV para escrita
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escreva o cabeçalho
        writer.writerow([
            'Chave', 'Sessao', 'Data', 'Oradores', 'Status'
        ])

        # Extraia dados do XML e escreva no CSV
        namespace = "{http://www.camara.sp.gov.br/}"
        for pauta in root.findall(f".//{namespace}Pauta"):
            chave = pauta.find(f"{namespace}Chave").text
            sessao = pauta.find(f"{namespace}Sessao").text
            data = pauta.find(f"{namespace}Data").text
            oradores = pauta.find(f"{namespace}Oradores").text
            status = pauta.find(f"{namespace}Status").text

            # Escreva a linha no CSV
            writer.writerow([chave, sessao, data, oradores, status])
else:
    print(f"Falha na solicitação: {response.status_code}")

In [ ]:
## Dados de vereadores em exercício

# Defina a URL e os cabeçalhos da solicitação
url = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.camara.sp.gov.br/VereadoresCMSP"
}

# Corpo da solicitação SOAP
body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <VereadoresCMSP xmlns="http://www.camara.sp.gov.br/" />
  </soap:Body>
</soap:Envelope>"""

# Envie a solicitação SOAP
response = requests.post(url, headers=headers, data=body)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
        # Salvar o conteúdo XML da resposta em um arquivo
    with open('response.xml', 'wb') as file:
        file.write(response.content)
    # Parse o conteúdo XML da resposta
    root = ET.fromstring(response.content)

else:
    print(f"Falha na solicitação: {response.status_code}")

In [ ]:
#criando os dataframes

audiencias = pd.read_csv(r'audiencias.csv')
assuntos = pd.read_csv(r'catalogo_de_assuntos.csv')
pauta23 = pd.read_csv(r'pautas_sessoes_plenarias23.csv')
pauta22 = pd.read_csv(r'pautas_sessoes_plenarias22.csv')
pauta24 = pd.read_csv(r'pautas_sessoes_plenarias24.csv')

audiencias = audiencias.loc[(audiencias['Ano'] >= 2021)]

In [ ]:
audiencias.head()

In [ ]:
audiencias['Local'].value_counts()

In [ ]:
pauta23['ano'] = 2023
pauta24['ano'] = 2024
pauta22['ano'] = 2022

pautas = pd.concat([pauta22, pauta24, pauta23], ignore_index=True)

### Projetos propostos

In [ ]:
projetos = pd.read_csv(r"C:\Users\Diogo\Downloads\Pesquisa - Projetos (1).csv") ##substituir (dados do portal splegisconsulta.saopaulo.sp.leg.br)

In [ ]:
projetos.head()

In [ ]:
# Função para extrair os nomes dos vereadores e dos partidos
def extrair_nomes_vereadores_e_partidos(entrada):
    # Expressão regular para capturar nomes precedidos de "Ver."
    padrao_vereadores = r'Ver\. ([\w\s]+)'
    nomes_vereadores = re.findall(padrao_vereadores, entrada)
    
    # Expressão regular para capturar os partidos entre parênteses
    padrao_partidos = r'\((.*?)\)'
    partidos = re.findall(padrao_partidos, entrada)
    
    return nomes_vereadores, partidos

projetos['vereadores_partidos'] = projetos['Promoventes'].apply(extrair_nomes_vereadores_e_partidos)

projetos[['nomes_vereadores', 'partidos']] = pd.DataFrame(projetos['vereadores_partidos'].tolist(), index=projetos.index)

df_explodido = projetos.explode(['nomes_vereadores', 'partidos'])
df_explodido.drop(columns=['vereadores_partidos', 'Promoventes'], inplace=True)

df_explodido.head()

partidos = df_explodido.drop(columns=['Ementa', 'Projeto', 'Norma', 'Palavras-chave'])
partidos['nome_vereador'] = partidos['nomes_vereadores']

# Criando nuvens de palavras

In [ ]:
!pip install wordcloud matplotlib

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
textos = ' '.join(df_explodido['Palavras-chave'].astype(str))

stopwords = set(STOPWORDS)
stopwords.update(["de", "municipal", 'paulo', 'sao', 'lei', 'parlamentar', 'frente', 'secretaria'])

wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(textos)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import pandas as pd
import re

df = pd.read_csv(r"C:\Users\Diogo\Downloads\Pesquisa - Projetos (1).csv")

# Função para extrair os nomes dos vereadores e remover prefixos como "dra."
def extrair_nomes_vereadores(entrada):
    # Expressão regular para capturar nomes precedidos de "Ver."
    padrao = r'Ver\. ([\w\s]+)'
    nomes = re.findall(padrao, entrada)
    # Remover prefixo "dra." dos nomes
    nomes = [re.sub(r'\bdra\.\s*', '', nome) for nome in nomes]
    return nomes

df['Promoventes'] = df['Promoventes'].apply(extrair_nomes_vereadores)

df_explodido = df.explode('Promoventes')

contagem_projetos = df_explodido['Promoventes'].value_counts()
print(contagem_projetos)

contagem_projetos_df = contagem_projetos.reset_index()
contagem_projetos_df.columns = ['nome_vereador', 'numero_projetos']
print(contagem_projetos_df)

In [ ]:
contagem_projetos_df.to_excel('projetos.xlsx')

In [ ]:
df_explodido=df_explodido.dropna(subset=['partidos'])
df_explodido.head()

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["de", "municipal", 'paulo', 'sao', 'lei', 'parlamentar', 'frente', 'da', 'e', 'secretaria'])

# Função para gerar e plotar a nuvem de palavras
def gerar_nuvem_palavras(texto, titulo, ax):
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(texto)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(titulo, fontsize=14)
    ax.axis('off')

# Agrupar os textos dos projetos de lei por partido
partidos = df_explodido['partidos'].unique()

# Configurar o tamanho da figura
fig, axes = plt.subplots(len(partidos), 1, figsize=(10, len(partidos) * 5))

# Gerar e plotar a nuvem de palavras para cada partido
for i, partido in enumerate(partidos):
    textos_partido = ' '.join(df_explodido[df_explodido['partidos'] == partido]['Palavras-chave'].astype(str))
    gerar_nuvem_palavras(textos_partido, partido, axes[i])

# Ajustar o layout
plt.tight_layout()
plt.show()

### Só com leis aprovadas

In [ ]:
projetos = pd.read_csv(r"C:\Users\Diogo\Downloads\Pesquisa - Projetos (2).csv") ##substituir (consulta no portal splegisconsulta.saopaulo.sp.leg.br)

In [ ]:
# Carregue seu dataset (substitua 'seu_arquivo.csv' pelo nome do seu arquivo)
df = pd.read_csv(r"C:\Users\Diogo\Downloads\Pesquisa - Projetos (2).csv")

# Função para extrair os nomes dos vereadores
def extrair_nomes_vereadores(entrada):
    # Expressão regular para capturar nomes precedidos de "Ver."
    padrao = r'Ver\. ([\w\s]+)'
    return re.findall(padrao, entrada)

df['Promoventes'] = df['Promoventes'].apply(extrair_nomes_vereadores)

df_explodido = df.explode('Promoventes')

contagem_projetos = df_explodido['Promoventes'].value_counts()
print(contagem_projetos)

contagem_projetos_aprov = contagem_projetos.reset_index()
contagem_projetos_aprov.columns = ['nome_vereador', 'numero_projetos']
print(contagem_projetos_aprov)

contagem_projetos_aprov.to_excel('projetos_aprovados.xlsx')

In [ ]:
partidos.head()

In [ ]:
projetos_final = pd.merge(contagem_projetos_aprov, contagem_projetos_df, on=['nome_vereador'])
projetos_final = pd.merge(partidos, projetos_final, on=['nome_vereador'])

In [ ]:
projetos_final.head()
projetos_final.to_excel('projetos.xlsx')

In [ ]:
# Função para extrair os nomes dos vereadores e dos partidos
def extrair_nomes_vereadores_e_partidos(entrada):
    # Expressão regular para capturar nomes precedidos de "Ver."
    padrao_vereadores = r'Ver\. ([\w\s]+)'
    nomes_vereadores = re.findall(padrao_vereadores, entrada)
    
    # Expressão regular para capturar os partidos entre parênteses
    padrao_partidos = r'\((.*?)\)'
    partidos = re.findall(padrao_partidos, entrada)
    
    return nomes_vereadores, partidos

# Aplique a função à coluna de nomes de vereadores
projetos['vereadores_partidos'] = projetos['Promoventes'].apply(extrair_nomes_vereadores_e_partidos)

# Separe os nomes dos vereadores e os partidos em colunas diferentes
projetos[['nomes_vereadores', 'partidos']] = pd.DataFrame(projetos['vereadores_partidos'].tolist(), index=projetos.index)

# Exploda a lista de nomes de vereadores em linhas separadas
df_explodido = projetos.explode(['nomes_vereadores', 'partidos'])
df_explodido.drop(columns=['vereadores_partidos', 'Promoventes'], inplace=True)

# Exibir o DataFrame com os vereadores e partidos
df_explodido.head()

partidos = df_explodido.drop(columns=['Ementa', 'Projeto', 'Norma', 'Palavras-chave'])
partidos['nome_vereador'] = partidos['nomes_vereadores']

In [ ]:
# Supondo que você tem uma coluna com textos dos projetos de lei, concatene todos os textos em uma única string
textos = ' '.join(df_explodido['Palavras-chave'].astype(str))

# Atualize as stopwords para incluir "de" e "municipal"
stopwords = set(STOPWORDS)
stopwords.update(["de", "municipal", 'paulo', 'sao', 'lei', 'parlamentar', 'frente', 'secretaria'])

# Crie a nuvem de palavras
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(textos)

# Exiba a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
df_explodido.tail()

Por vereador

In [ ]:
vespoli = df_explodido.loc[(df_explodido['nomes_vereadores'] == 'PROFESSOR TONINHO VESPOLI ')]
messias = df_explodido.loc[(df_explodido['nomes_vereadores'] == 'MARCELO MESSIAS ')]

In [ ]:
messias.head()

In [ ]:
#Vespoli

textos = ' '.join(vespoli['Palavras-chave'].astype(str))

stopwords = set(STOPWORDS)
stopwords.update(["de", "municipal", 'paulo', 'sao', 'lei', 'parlamentar', 'frente', 'secretaria', 'da'])

wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(textos)

# Exiba a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#Messias

textos = ' '.join(messias['Palavras-chave'].astype(str))

stopwords = set(STOPWORDS)
stopwords.update(["de", "municipal", 'paulo', 'sao', 'lei', 'parlamentar', 'frente', 'secretaria', 'da'])

wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(textos)

# Exiba a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### Dados de candidatos

In [ ]:
import pandas as pd
cand = pd.read_csv(r"C:\Users\Diogo\Documents\consulta_cand_2020_BRASIL.csv", encoding='latin1', sep=';') #substituir (dados de dadosabertos.tse.jus.br)

In [ ]:
colunas_desejadas = ['DS_CARGO', 'NM_UE', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'SG_PARTIDO', 'CD_MUNICIPIO_NASCIMENTO', 'NR_IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 'DS_COR_RACA', 'VR_DESPESA_MAX_CAMPANHA', 'ST_REELEICAO', 'DS_SITUACAO_CANDIDATO_TOT']
cand = cand[colunas_desejadas]

In [ ]:
cand = cand.loc[(cand['DS_CARGO'] == 'VEREADOR')]
cand = cand.loc[(cand['NM_UE'] == 'SÃO PAULO')]

In [ ]:
cand.head()

In [ ]:
#total buscando reeleição
cand['ST_REELEICAO'].value_counts()

In [ ]:
#total por partido
cand['SG_PARTIDO'].value_counts()

In [ ]:
bens = pd.read_csv(r"C:\Users\Diogo\Documents\bem_candidato_2020_BRASIL.csv", sep=';', encoding='latin1', decimal=',', thousands='.') #substituir (dados de dadosabertos.tse.jus.br)

In [ ]:
bens = bens.loc[(bens['NM_UE'] == 'SÃO PAULO')]

In [ ]:
bens_agrupado = bens.groupby('SQ_CANDIDATO')['VR_BEM_CANDIDATO'].sum().reset_index()

# Renomeie a coluna 'valor_bem' para 'valor_total_bens' para maior clareza
bens_agrupado = bens_agrupado.rename(columns={'valor_bem': 'valor_total_bens'})

In [ ]:
bens_agrupado['sq_candidato'] = bens_agrupado['SQ_CANDIDATO']
bens_agrupado.head()

In [ ]:
resultado = pd.read_csv(r"C:\Users\Diogo\Downloads\votacao_candidato-vereador_2020_sudeste_sp_sao_paulo.csv\votacao_candidato-vereador_2020_sudeste_sp_sao_paulo.csv", encoding='latin1', sep=';')
cand['nm_candidato'] = cand['NM_CANDIDATO']
cand_eleito = pd.merge(resultado, cand, on=['nm_candidato'])

In [ ]:
cand_eleito = cand_eleito.loc[(cand_eleito['ds_sit_totalizacao'] == 'Eleito')]

In [ ]:
cand_eleito['ST_REELEICAO'].value_counts()

In [ ]:
cand_eleito.describe()

In [ ]:
cand_eleito['SG_PARTIDO'].value_counts()

In [ ]:
cand_eleito.loc[(cand_eleito['ST_REELEICAO'] == 'S')]

In [ ]:
cand_eleito = pd.merge(cand_eleito, bens_agrupado, on=['sq_candidato'])

In [ ]:
cand_eleito['VR_BEM_CANDIDATO'].mean()

In [ ]:
bens_agrupado['VR_BEM_CANDIDATO'].mean()

In [ ]:
despesas = pd.read_csv(r"C:\Users\Diogo\Documents\despesas.csv", encoding='latin1')
despesas = despesas.loc[(despesas['NM_UE'] == 'SÃO PAULO')]
despesas

In [ ]:
despesas = despesas.groupby('SQ_CANDIDATO')['VR_DESPESA_CONTRATADA'].sum().reset_index()
despesas

In [ ]:
despesas['VR_DESPESA_CONTRATADA'].mean()

In [ ]:
despesas['sq_candidato'] = despesas['SQ_CANDIDATO']
despesas_eleito = pd.merge(cand_eleito, despesas, on=['sq_candidato'])
despesas_eleito['VR_DESPESA_CONTRATADA'].mean()